In [21]:
!pip freeze | grep scikit-learn

scikit-learn==1.2.2


In [22]:
import pickle
import pandas as pd
import statistics

In [23]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [24]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [25]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet')

In [26]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [27]:
sd = round(statistics.stdev(y_pred), 2)

## Q1. Notebook
What's the standard deviation of the predicted duration for the February 2022 dataset?

In [28]:
print(f"Answer: The standard deviation is {sd}.")

Answer: The standard deviation is 5.28.


## Q2. Preparing the output
What's the size of the output file?

In [34]:
year = 2022
month = 2

df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [35]:
df_result = pd.DataFrame()
df_result['ride_id'] = df['ride_id']
df_result['predicted_duration'] = y_pred

output_file = f"output/ride_prediction_{year:04d}-{month:02d}.parquet"

df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [37]:
!ls -lh output/

total 117184
-rw-r--r--  1 toph  staff    57M 24 Jun 00:40 ride_prediction_2022-02.parquet


Answer: The size of the output file is 57M.

## Q3. Creating the scoring script
Now let's turn the notebook into a script.  
Which command you need to execute for that?

In [38]:
!jupyter nbconvert --to script starter_week4.ipynb

[NbConvertApp] Converting notebook starter_week4.ipynb to script
[NbConvertApp] Writing 1797 bytes to starter_week4.py


## Q4. Virtual environment
What's the first hash for the Scikit-Learn dependency?  
  
Answer: sha256:065e9673e24e0dc5113e2dd2b4ca30c9d8aa2fa90f4c0597241c93b63130d233

## Q5. Parametrize the script
Run the script for March 2022.  
  
What's the mean predicted duration?

In [40]:
year = 2022
month = 3

# input_file = f'https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_{year:04d}-{month:02d}.parquet'

In [45]:
df = read_data('../data/yellow_tripdata_2022-03.parquet')
dicts = df[categorical].to_dict(orient='records')

X_val = dv.transform(dicts)
y_pred = model.predict(X_val)
res = round(y_pred.mean(),2)

print(f"The Answer is {res}")

The Answer is 12.76


## Q6 Docker container
What's the mean predicted duration for April 2022?

In [51]:
year = 2022
month = 4

# input_file = f'https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_{year:04d}-{month:02d}.parquet'

In [52]:
df = read_data('../data/yellow_tripdata_2022-04.parquet')
dicts = df[categorical].to_dict(orient='records')

X_val = dv.transform(dicts)
y_pred = model.predict(X_val)
res = round(y_pred.mean(),2)

print(f"The Answer is {res}")

The Answer is 12.87
